In [1]:
############  FUNCTION TO PARSE ICE HOCKEY DATA  ############
############ FIRST BOOK USING THE METHOD OF REQUESTING AND ENTIRE YEAR OF DATA AT ONCE ############




In [2]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
import time
import datetime
import os
import tqdm as tqdm

In [3]:
### Function to create the URLS to request a full year of data

## static peices of the url
url_a = 'https://my.mhsaa.com/DesktopModules/MHSAA-Endpoint/handlers/ContestWithParticipantDepotSearch.ashx?Method=ScoreCenterSearch&StartDate='
url_b = '&EndDate='
url_c = '&Sport='
url_d = '&Level=V'

# Sport code 
sport = 'BIH'

# construct the url for a given year
def url_construct(year):
    # Create start and end dates in teh YYYY-MM-DD format
    # Request one academic year of data at a time August 1st to July 31st
    start_date = str(year) + '-08-01'
    end_date = str(year + 1) + '-07-31'
    url = url_a + start_date + url_b + end_date + url_c + sport + url_d
    return url

# extrtact academic year from date
def get_year(date):
    return date[:4]


# If tournament name contains Regional, then regional; if quarterfinal, then quarterfinal, ect
def get_postseason_level(tournament_name):
    if tournament_name is not None:
        if 'Regional' in tournament_name:
            return 'Regional'
        elif 'Quarterfinal' in tournament_name:
            return 'Quarterfinal'
        elif 'Semifinal' in tournament_name:
            return 'Semifinal'
        elif 'Final' in tournament_name:
            return 'Final'
    return ''


In [4]:
#### Transformations of the data by function

def parse_json(data):
    # create a new column to hold the OT info
    df['OT'] = df['Score'].str.extract(r'(\(.*\))')
    ## clean up the column by removing the parentheses
    df['OT'] = df['OT'].str.replace('(', '')
    df['OT'] = df['OT'].str.replace(')', '')
    # get and save the acadeic year
    df['SeasonYear'] = year
    # create column with postseason level
    df['PostSeasonLevel'] = df['TournamentName'].apply(get_postseason_level)




# create a list of columns to keep
keep_cols = ['HostSchoolId', 'HostTeamName', 'HostScore', 'HostRecord',
             'AwaySchoolId', 'AwayTeamName', 'AwayScore', 'AwayRecord',
             'OT',
                        

             'ContestName', 'StartDate', 'SeasonYear',
             'ContestType', 'IsPostSeason',
             'PostSeasonLevel', 'TournamentNumber', 'TournamentName',
             

             'PostSeasonInfo', 'Notes']


In [14]:
## Create a loop to set a range of years to request data for and store in a single dataframe

# create an empty dataframe to store the data
all_hockey = pd.DataFrame()

# set the range of years to request data for
years = range(2009, 2023)

# loop through the years and request the data
for year in tqdm.tqdm(years):
    # construct the url for the year
    url = url_construct(year)
    # request the data
    response = requests.get(url)
    # convert the response to json
    data = response.json()
    # convert the json to a dataframe
    df = pd.DataFrame(data)
    # parse the data
    parse_json(df)
    # append the data to the dataframe
    all_hockey = all_hockey.append(df[keep_cols])

  0%|          | 0/14 [00:00<?, ?it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_6012\1866463018.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['OT'] = df['OT'].str.replace('(', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_6012\1866463018.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['OT'] = df['OT'].str.replace(')', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_6012\1892256670.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_hockey = all_hockey.append(df[keep_cols])
  7%|▋         | 1/14 [00:02<00:36,  2.82s/it]C:\Users\Justin\AppData\Local\Temp\ipykerne

In [15]:
all_hockey.describe()

all_hockey.columns
all_hockey.head()


all_hockey.sample(30)
len(all_hockey)

21913

In [16]:
### Save the data to a csv file


# set the path to save the data
# path = os.path.join('..', 'data', 'ice_hockey', 'ice_hockey_raw_2000_2023.csv')

# save the data
all_hockey.to_csv(path, index=False)
